In this example, we will assess the risk of a student failing a course module based on student characterstics (gender, age, etc.) and information about their activity (studied credits, number of previous attempts to pass the course). To do that, we will train a student model using logistic regression.

Then, we will try to improve the model's performance in terms of accuracy by using the assignments' grades as an additional factor.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#read student information
studentInfo = pd.read_csv("/kaggle/input/open-university-learning-analytics-dataset/anonymiseddata/studentInfo.csv")
vle = pd.read_csv("/kaggle/input/open-university-learning-analytics-dataset/anonymiseddata/vle.csv")
#print out the 10 first rows of the data
studentInfo.head(10)


In [ ]:
studentInfo["disability"].unique()

In [ ]:
studentInfo["imd_band"].unique()

In [ ]:
# lets see all potential final results
studentInfo["final_result"].unique()

In [ ]:
#create a new column to classify final results. classify studets with a pass or distinction as "1", the rest as "0"
studentInfo["result.class"] = 1

#studentInfo["result.class"] = studentInfo["final_result"].apply(lambda x: 0 if (x == 'Fail') | x == "Withdrawn") else 1)
studentInfo["result.class"].loc[(studentInfo["final_result"] == "Withdrawn") | (studentInfo["final_result"] == "Fail")] = 0
studentInfo["result.class"].loc[(studentInfo["imd_band"] == "90-100%") | (studentInfo["imd_band"] == "80-90%") | (studentInfo["imd_band"] == "70-80%") | (studentInfo["imd_band"] == "nan")] = 0
studentInfo["result.class"].loc[(studentInfo["disability"] == "N")] = 0
#and look at the dataset again
studentInfo.head()

We create one dataframe (Xfactors) with all the factors (variables) that we will use to assess whether a student will pass of fail the course

In [ ]:
Xfactors = studentInfo[["gender", "region", "highest_education", "imd_band", "age_band", "num_of_prev_attempts", "studied_credits", "disability"]]
X_noncat = pd.get_dummies(Xfactors)

X_noncat.head(5)

Then we create another variable (Youtcome) which represents the outcome, that is what we want to assess. Here, we want to assess whether a student will pass the course successfully or not - which is represented by the variable "result.class". Please remember, 1 means the student passes the course, 0 means the student fails the course.

In [ ]:
Youtcome = studentInfo["result.class"].values
Youtcome

Now its time to fit our model! This means that we will use "old" data - where we already know the outcome - to train the model. We will also keep a part of the old data to test our model's performance - that is whether the model learned to an acceptable degree to assess student performance.
The datasets used for training have the suffix "_train" while the datasets saved for testing have the suffix "_test".
The model is trained as a logistic regression binary classifier.

In [ ]:
#fit the model
X_train, X_test, y_train, y_test = train_test_split(X_noncat, Youtcome, test_size=0.3, random_state=0)
OurModel = LogisticRegression()
OurModel.fit(X_train, y_train)

Now we will use our model (OurModel) to assess student performance using the test dataset.

In [ ]:
#predict on a testset
y_pred = OurModel.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(OurModel.score(X_test, y_test)))

As you can see from the results, our model can assess student performance correctly with a 61% accuracy. 
This is not really good, is it? 
Lets try to improve the accuracy by adding one more variable to the predictive features (the Xfactors dataframe): the students'average grade of the Teacher Marked Asssessments (TMA) assignments of the course.

To do that, we will need the data contained in the tables: assessment and studentAssessment.
The analysis follows.

In [ ]:
#read additional data
studentAssessments = pd.read_csv("/kaggle/input/open-university-learning-analytics-dataset/anonymiseddata/studentAssessment.csv")
assessments = pd.read_csv("/kaggle/input/open-university-learning-analytics-dataset/anonymiseddata/assessments.csv")

#retrieve the ids only of the teacher assessments (TMA)
TAM = assessments.loc[assessments['assessment_type'] == "TMA"]


#then keep the students assessments (grades) that were only given by the teacher (TAM) and remove unknown entries ("?")
TAM_student_grades = studentAssessments.loc[studentAssessments.id_assessment.isin(TAM["id_assessment"])]
TAM_student_grades = TAM_student_grades.loc[TAM_student_grades['score'] != '?']

In [ ]:
#create an empty list where we will save the average grade for each and every student
avg_grades = [] 
#for each student find all TMA scores for the course we are interested, and get the mean value

for i in range (0, len(studentInfo['id_student'])):
    
    this_student = studentAssessments.loc[(studentAssessments['id_student'] == studentInfo['id_student'][i]) &
                                          (studentAssessments['score'] != '?')]
    
    assmt = list(this_student['id_assessment'])
    score = list(this_student['score'])
                 
    #score = list(this_student['score'].astype(float))
    
    final_score = 0
    for j in range(0, len(assmt)):
        idx = assessments.loc[assessments.id_assessment == assmt[j]].index[0]
        if((assessments.code_module[idx] == studentInfo['code_module'][i]) & (assessments.assessment_type[idx] == "TMA")):
            final_score = final_score + (float(assessments.weight[idx])*score[j])/100
            
    avg_grades.append(final_score)
    
#add the new information about average TAM grades to the student information dataframe

studentInfo['avg_TMA_assessment'] = avg_grades

In [ ]:
#add the new information about average TAM grades to our model

Xfactors_updated = studentInfo[["gender", "region", "highest_education", "imd_band", "age_band", "num_of_prev_attempts", "studied_credits", "disability", "avg_TMA_assessment"]]
X_noncat_updated = pd.get_dummies(Xfactors_updated)
X_noncat_updated = X_noncat_updated.fillna(0)
X_noncat_updated.head(5)

In [ ]:
#fit the model again
X_train, X_test, y_train, y_test = train_test_split(X_noncat_updated, Youtcome, test_size=0.3, random_state=0)
OurModelUpdated = LogisticRegression()
OurModelUpdated.fit(X_train, y_train)

Now we will use the updated model (OurModelUpdated) to assess student performance using the test dataset.

In [ ]:
#predict on a testset
y_predUpdated = OurModelUpdated.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(OurModelUpdated.score(X_test, y_test)))

Well, what about that! The model now can assess students' performance with 83% accuracy!
Looks like the grades of Teacher Marked Assignments really helped us to improve the performance of our model :)
I wonder what else could help.... ;)

How to evaluate the models and choose "THE BEST"?

The answer to this question is a bit like "42" (if you don't know what "42" is, look here: https://en.wikipedia.org/wiki/Phrases_from_The_Hitchhiker%27s_Guide_to_the_Galaxy#Answer_to_the_Ultimate_Question_of_Life,_the_Universe,_and_Everything_(42))

The important thing here is not the answer, but the QUESTION. How do we define "the best"? The "best" in terms of what? And is even our best, good enough (and how to deal with it if not)?

Lets say that we need to choose "the best" from the two models we built last week: OurModel and OurModelUpdated (amazing names, i know.....)

How can we do that?

**Confusion Matrix**
A confusion matrix is a table that is often used to describe the performance of a classification model - like our model! The confusion matrix allows us to identify which classes were predicted correctly and which ones were misspredicted - for example, if our model tends to identify students as at risk while they actually are not. You can read more about confusion matrices, here:

https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

In [ ]:
#in our example...
#lets calculate the confusion matrix for the first model: OurModel
from sklearn.metrics import confusion_matrix

OurModelCM = confusion_matrix(y_test, y_pred)
print(OurModelCM)

The results above show the following: a) 3390 people were predicted as failing the course and they indeed failed b) 1727 people were predicted as passing the course and they failed c) 2081 people were predicted as failing the course and they passed d) 2580 people were predicted as passing the course and they indeed passed

Overall, 1727 + 2081 = 3808 people were misclassified! (not good...)

In [ ]:
#now lets calculate the confusion matrix for the second model: OurModelUpdated

OurModelUpdatedCM = confusion_matrix(y_test, y_predUpdated)
print(OurModelUpdatedCM)

The results above show the following: a) 4354 people were predicted as failing the course and they indeed failed b) 763 people were predicted as passing the course and they failed c) 866 people were predicted as failing the course and they passed d) 3795 people were predicted as passing the course and they indeed passed

Overall, 763 + 866 = 1629 people were misclassified! (better than before but is if acceptable?)

**Precision, recall, F-measure and support metrics**
The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier to not label a sample as positive if it is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.

The F-beta score weights the recall more than the precision by a factor of beta. beta = 1.0 means recall and precision are equally important.

The support is the number of occurrences of each class in y_test.

In [ ]:
from sklearn.metrics import classification_report
#compute the above metrics for the first model (OurModel)
print(classification_report(y_test, y_pred))

In [ ]:
#compute the above metrics for the second model (OurModelUpdated)
print(classification_report(y_test, y_predUpdated))

**ROC curves**
The receiver operating characteristic (ROC) curve is another common tool used with binary classifiers. The dotted line represents the ROC curve of a purely random classifier; a good classifier stays as far away from that line as possible (toward the top-left corner).

In [ ]:
#try out the ROC curve of the first model (OurModel)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt 

X_train, X_test, y_train, y_test = train_test_split(X_noncat, Youtcome, test_size=0.3, random_state=0)
OurModel = LogisticRegression()
OurModel.fit(X_train, y_train)

logit_roc_auc = roc_auc_score(y_test, OurModel.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, OurModel.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for Original Model')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
#try out the ROC curve of the second model (OurModelUpdated)

X_train, X_test, y_train, y_test = train_test_split(X_noncat_updated, Youtcome, test_size=0.3, random_state=0)
OurModelUpdated = LogisticRegression()
OurModelUpdated.fit(X_train, y_train)

logit_roc_auc = roc_auc_score(y_test, OurModelUpdated.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, OurModelUpdated.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for Updated Model')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

**Last thing to discuss...**

I've tried to demontstrate a few alternatives that can help you choose the "best model" for the needs of the second assignment but also for our own research :)

Please let me know your thoughts, questions or ideas about this topic!

Cheers :)